In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from random import randint
import random
import numpy as np
import pandas as pd 
import torch.optim as optim
import os
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
import torch
from scipy.stats import vonmises
import numpy.fft as fft 
res = 8

In [86]:
##Define the network to train on the data 
class CircleNet(nn.Sequential): 
    def __init__(self, res, dropout, n_out): 
        super(CircleNet, self).__init__()
        self.n_in = res
        self.affine1 = nn.Linear(self.n_in, 6)
        self.drop1 = nn.Dropout(dropout)
        ##self.affine2 = nn.Linear(8, 4)
        ##self.drop2 = nn.Dropout(dropout)
        self.affine3 = nn.Linear(6, n_out)
    def forward(self, x): 
        x = F.relu(self.affine1(x))
        x = self.drop1(x)
        ##x = F.relu(self.affine2(x))
        ##x = self.drop2(x)
        x = F.relu(self.affine3(x))
        x = F.softmax(x, dim = 1)
        return x

This is the Data Generation Code


In [50]:
## make some uniform circular data
res = 8
num_points = 10000
num_samples = 100
data = np.zeros([num_points,res + 1])
for i in range(num_points): 
    for j in range(num_samples): 
        num = randint(0,res-1)
        data[i][num]+= 1
        data[i][res] = -1
    data[i][:-1] = (data[i][:-1]-12)/5
np.savetxt("uniform8ptdata100.csv", data, delimiter = ",")

In [37]:
data = np.zeros([10000,res+1])
for i in range(10000):
    SAMPLE_SIZE = 100
    kappa = 1.5
    rotate_amt = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
    for l in range(SAMPLE_SIZE):
        random_rotate_factor = randint(0,1)
        vm_dist[l] = ((vm_dist[l] + np.pi + np.pi*random_rotate_factor)/(2*np.pi))%1*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - 12)/5
    data[i,-1] = -6
np.savetxt("bimodal8pt100.csv", data, delimiter = ",")    

In [83]:
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1.5
    rotate_amt = random.random()*2*np.pi
    vm_dist = vonmises.rvs(kappa, loc = rotate_amt, size=SAMPLE_SIZE)
   
      
    vm_dist = ((vm_dist + np.pi)/(2*np.pi))%1*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    ##rotate_amt = randint(0,res - 1)
    ##data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i][:-1] = (data[i][:-1] - 12)/5
    data[i,-1] = -3
np.savetxt("vonmises8ptnormalizedtest.csv", data, delimiter = ",")    

In [105]:
print(data[7])

[15. 14.  4. 18. 15. 11.  9. 14. -6.]


In [3]:
##This is a class that allows us to load data from csv files and make them into datasets that the dataloader can use
class CircularDataset(Dataset): 
    def __init__(self, data_root): 
        self.data = []
        self.dist_coder = LabelEncoder()
        self.dist_list = []
        for path in data_root: 
            array = pd.read_csv(path)
            
            self.dist_list += [array.values[0,-1]] ##get the distribution type for each file and add it to the list
            if len(self.data) == 0: 
                self.data = array.values
            else:
                print(self.data.shape)
                print(array.values.shape)
                self.data = np.concatenate((self.data,array.values), axis = 0)
        self.dist_coder.fit(self.dist_list)
    def __getitem__(self,idx): 
        return self.data[idx][0:-1], self.to_one_hot(self.dist_coder,([self.data[idx,-1]]))[0]
    def __len__(self): 
        return len(self.data)
    def to_one_hot(self, codec, values):
        value_idxs = codec.transform(values)
        return torch.eye(len(codec.classes_))[value_idxs]

This is the code where we train the network on the data

In [90]:
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.05)
rate = .005
batchsize = 100
dataset = CircularDataset(["uniform8ptdata100.csv", "bimodal8pt100.csv","vonmises8ptnormalized.csv"])
testdata = CircularDataset(["uniform8ptdata100test.csv", "bimodal8pt100test.csv","vonmises8ptnormalizedtest.csv"])
iterator = torch.utils.data.DataLoader(dataset, batch_size = batchsize, shuffle = True)
testiterator = torch.utils.data.DataLoader(testdata, batch_size = 1, shuffle = True)
net = CircleNet(8,0,3)
optimizer = optim.Adam(net.parameters(),lr = rate)
running_loss = 0
epochs = 10
net.apply(init_weights)
net.train()
loss_func = nn.CrossEntropyLoss()
for e in range(epochs): 
    print(running_loss)
    running_loss = 0
    accuracy = 0
    i = 0
    for dist, labels in iterator: 
        
        
        optimizer.zero_grad()
        out = net(dist.float())
        
        loss = loss_func(out,torch.max(labels, 1)[1])
       
           
        loss.backward()
        optimizer.step()
        running_loss += loss
    for dist, labels in testiterator: 
        ps = net(dist.float())
        
        i += 1
        top_p, top_class = ps.topk(1, dim=1)
       
        top_p2, top_class2 = labels.topk(1, dim = 1)
        
        equals = top_class == top_class2
       
        accuracy += torch.sum(equals.float())
        
    
    print(accuracy)
    print(i)
    accuracy = 0
    

(9999, 9)
(9999, 9)
(19998, 9)
(9999, 9)
(99, 9)
(99, 9)
(198, 9)
(99, 9)
0
tensor(215.)
297
tensor(296.0301, grad_fn=<AddBackward0>)
tensor(245.)
297
tensor(243.9988, grad_fn=<AddBackward0>)
tensor(245.)
297
tensor(233.5331, grad_fn=<AddBackward0>)
tensor(249.)
297
tensor(229.1032, grad_fn=<AddBackward0>)
tensor(261.)
297
tensor(211.3859, grad_fn=<AddBackward0>)
tensor(264.)
297
tensor(207.6109, grad_fn=<AddBackward0>)
tensor(263.)
297
tensor(206.1604, grad_fn=<AddBackward0>)
tensor(264.)
297
tensor(205.1253, grad_fn=<AddBackward0>)
tensor(262.)
297
tensor(204.4244, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [96]:
testdata = CircularDataset(["bimodal8pt100test.csv"])
testiterator = torch.utils.data.DataLoader(testdata, batch_size = 1, shuffle = True)
accuracy = 0

i = 0
for dist, labels in testiterator: 
    ps = net(dist.float()/20)
        
    print(dist)
    i+=1
    top_p, top_class = ps.topk(1, dim=1)
    
    top_p2, top_class2 = labels.topk(1, dim = 1)
        
    equals = top_class == top_class2
   
    accuracy += torch.sum(equals.float())
        
print(accuracy/99)
print(i)


tensor([[ 0.0000,  1.0000,  1.0000, -1.0000,  1.0000,  0.0000,  0.0000, -1.2000]],
       dtype=torch.float64)
tensor([[-0.6000,  0.0000,  1.6000, -0.2000, -1.0000,  1.0000,  0.4000, -0.4000]],
       dtype=torch.float64)
tensor([[ 1.4000,  1.4000, -0.2000, -0.4000, -0.6000,  0.8000, -0.8000, -0.8000]],
       dtype=torch.float64)
tensor([[-0.4000, -0.2000, -0.2000,  0.0000, -0.2000,  0.6000,  1.2000,  0.0000]],
       dtype=torch.float64)
tensor([[ 1.6000, -0.8000, -0.8000, -0.4000,  1.6000, -0.8000, -0.2000,  0.6000]],
       dtype=torch.float64)
tensor([[ 0.4000, -0.6000, -0.6000, -0.2000,  0.0000, -0.4000,  1.2000,  1.0000]],
       dtype=torch.float64)
tensor([[ 0.2000,  0.0000,  0.6000,  0.4000,  0.0000,  1.2000, -0.6000, -1.0000]],
       dtype=torch.float64)
tensor([[ 1.0000,  0.4000, -0.4000,  0.0000,  1.0000,  0.2000, -0.6000, -0.8000]],
       dtype=torch.float64)
tensor([[ 0.4000,  0.6000,  1.0000,  0.8000, -1.0000,  0.8000, -0.2000, -1.6000]],
       dtype=torch.float64)
t

       dtype=torch.float64)
tensor([[ 0.4000,  1.6000, -0.6000, -1.2000, -0.2000,  2.2000, -0.6000, -0.8000]],
       dtype=torch.float64)
tensor([[ 1.6000, -1.2000,  0.4000,  1.0000,  0.0000, -1.2000, -1.0000,  1.2000]],
       dtype=torch.float64)
tensor([[ 0.0000,  1.6000, -0.2000, -1.2000,  0.2000,  0.8000,  0.0000, -0.4000]],
       dtype=torch.float64)
tensor([[-0.4000, -0.6000,  1.0000,  0.2000, -0.2000, -0.6000,  0.2000,  1.2000]],
       dtype=torch.float64)
tensor([[-0.2000,  1.4000,  0.0000, -0.6000,  0.8000,  1.0000, -0.6000, -1.0000]],
       dtype=torch.float64)
tensor([[ 0.6000,  0.0000,  1.2000, -1.4000, -1.0000,  0.6000,  0.8000,  0.0000]],
       dtype=torch.float64)
tensor([[-1.2000,  1.2000,  0.8000, -1.0000, -0.8000,  1.8000,  1.0000, -1.0000]],
       dtype=torch.float64)
tensor([[-0.2000,  2.4000, -0.4000, -0.2000, -2.0000,  0.6000,  0.8000, -0.2000]],
       dtype=torch.float64)
tensor([[ 1.6000, -0.8000, -0.2000, -0.6000,  0.4000,  0.6000, -1.6000,  1.4000]],
 

In [ ]:
def rotate(vec, num_rotations): 
    vec2 = np.zeros(len(vec))
    for i in range(len(vec)): 
        vec2[(i + num_rotations )% len(vec)] = vec[i]
    return vec2
data = np.zeros([100,res+1])
for i in range(100):
    SAMPLE_SIZE = 100
    kappa = 1
    vm_dist = vonmises.rvs(kappa, size=SAMPLE_SIZE)
    vm_dist = (vm_dist + np.pi)/(2*np.pi)*(res*10)//10
    
    for k in range(SAMPLE_SIZE):
        data[i][int(vm_dist[k])] += 1
    
    rotate_amt = randint(0,res - 1)
    data[i,0:-1] = rotate(data[i,0:-1], rotate_amt)
    data[i,-1] = -3
np.savetxt("vonmises8ptdata100test.csv", data, delimiter = ",")    

In [92]:
paramlist2 = []
for parameter in net.parameters(): 
    paramlist2 += [parameter]

In [98]:
savednet2 = net

In [93]:
array2 = paramlist2[0].detach().numpy()
array2

array([[-1.5944755 , -1.6633469 ,  0.5776691 ,  1.0879722 , -0.65692437,
        -0.71620417,  1.2153625 ,  0.8287365 ],
       [ 0.6978802 , -0.48027298, -2.1328144 , -1.0842304 , -0.2966838 ,
         0.24222182,  0.87208647,  1.3551033 ],
       [-1.4021107 , -0.6938992 , -0.00644462,  0.8420382 ,  1.0692444 ,
         0.8139301 ,  0.4504028 , -0.3948431 ],
       [ 1.5802271 , -0.2276735 , -0.8352513 ,  0.8565605 ,  1.6216164 ,
        -1.4878672 , -2.996369  , -0.17060782],
       [-0.97879785,  0.8147126 , -0.32629663, -1.7353756 ,  0.06200669,
         1.6699992 ,  0.02718284, -1.9329549 ],
       [ 0.42568463,  0.0226077 , -0.08336811,  0.6659003 ,  1.0164266 ,
         0.7175958 ,  0.6100482 ,  0.73010075]], dtype=float32)

In [94]:
np.around(np.matmul(array2,fft.ifft(np.identity(8))),1)

array([[-0.1+0.j , -0.2-0.1j, -0.5-0.5j, -0. +0.j ,  0. +0.j , -0. -0.j ,
        -0.5+0.5j, -0.2+0.1j],
       [-0.1+0.j ,  0.3-0.7j,  0.2-0.1j, -0. +0.1j, -0.1+0.j , -0. -0.1j,
         0.2+0.1j,  0.3+0.7j],
       [ 0.1+0.j , -0.6-0.1j, -0.1-0.j , -0.1+0.j , -0.1+0.j , -0.1-0.j ,
        -0.1+0.j , -0.6+0.1j],
       [-0.2+0.j ,  0. +0.5j,  0.9-0.3j, -0. -0.1j,  0. +0.j , -0. +0.1j,
         0.9+0.3j,  0. -0.5j],
       [-0.3+0.j , -0.2-0.1j, -0.1+0.8j, -0. -0.j , -0. +0.j , -0. +0.j ,
        -0.1-0.8j, -0.2+0.1j],
       [ 0.5+0.j , -0.1-0.2j,  0.1-0.1j, -0. +0.j , -0. +0.j , -0. -0.j ,
         0.1+0.1j, -0.1+0.2j]])

In [77]:
fft.irfft(np.identity(8),8)

array([[ 0.125    ,  0.125    ,  0.125    ,  0.125    ,  0.125    ,
         0.125    ,  0.125    ,  0.125    ],
       [ 0.25     ,  0.1767767,  0.       , -0.1767767, -0.25     ,
        -0.1767767,  0.       ,  0.1767767],
       [ 0.25     ,  0.       , -0.25     ,  0.       ,  0.25     ,
         0.       , -0.25     ,  0.       ],
       [ 0.25     , -0.1767767,  0.       ,  0.1767767, -0.25     ,
         0.1767767,  0.       , -0.1767767],
       [ 0.125    , -0.125    ,  0.125    , -0.125    ,  0.125    ,
        -0.125    ,  0.125    , -0.125    ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ],
       [ 0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ]])

In [19]:
paramlist2

[Parameter containing:
 tensor([[ 1.8267,  1.6260,  1.0317,  0.1149, -1.1928, -2.3556, -1.1457,  1.0627],
         [-2.6579, -1.6477,  0.0733,  1.0506,  1.7144,  2.1449,  1.0407, -1.2851]],
        requires_grad=True), Parameter containing:
 tensor([0.2029, 0.3341], requires_grad=True), Parameter containing:
 tensor([[ 4.2196,  3.8495],
         [-2.9708, -2.5775]], requires_grad=True), Parameter containing:
 tensor([-6.3560, 11.3028], requires_grad=True)]

In [24]:
net(torch.from_numpy(dataset.__getitem__(1)[0]).float()/20)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [99]:
torch.save(savednet2.state_dict(), "8x6x2bianduniexample.pt")

0.1